# Tutorial for computing and reading emission lines.

In [1]:
import numpy as np

In [2]:
# Load GalacticusHDF5 class for reading/writing of Galacticus HDF5 files.
from galacticus.io import GalacticusHDF5
# Load emission line class to read/compute emission line luminosities. Pass Galacticus HDF5 object to emission
# line class.
from galacticus.EmissionLines import GalacticusEmissionLine

In [3]:
# Load Galacticus HDF5 file
ifile = "/Users/amerson/Data/Galacticus_Out/v0.9.4/wfirstSED_singleTree_lightcone/galacticus.hdf5"
GH5 = GalacticusHDF5(ifile,'r')

In [4]:
# Print available redshifts
for i in range(len(GH5.outputs.z)):
    print(str(GH5.outputs.iout[i])+" "+GH5.outputs.name[i]+" z = "+str(GH5.outputs.z[i]))

1 Output1 z = 2.0
2 Output2 z = 1.8
3 Output3 z = 1.6
4 Output4 z = 1.4
5 Output5 z = 1.2
6 Output6 z = 1.0


In [5]:
# Pass Galacticus HDF5 class onject to emission lines object.
EMLINE = GalacticusEmissionLine(GH5)

In [6]:
# Print list of emission lines that could be computed and rest wavelength from CLOUDY assigned to each line.
linesAvailable = EMLINE.getLineNames()
for line in linesAvailable:
    print(line+", rest wave. = "+str(EMLINE.getWavelength(line))+" Ang.")

balmerAlpha6563, rest wave. = 6566.81779348 Ang.
balmerBeta4861, rest wave. = 4864.28871684 Ang.
nitrogenII6548, rest wave. = 6551.95401396 Ang.
nitrogenII6584, rest wave. = 6587.97873113 Ang.
oxygenII3726, rest wave. = 3728.24052116 Ang.
oxygenII3729, rest wave. = 3731.24602252 Ang.
oxygenIII4959, rest wave. = 4961.99273106 Ang.
oxygenIII5007, rest wave. = 5010.00436409 Ang.
sulfurII6716, rest wave. = 6720.04321935 Ang.
sulfurII6731, rest wave. = 6735.03356571 Ang.


In [7]:
# In a raw Galacticus HDF5 output no emission line luminosities are available, so they need to be computed. We 
# can comute them using the getLineLuminosity function in the GalacticusEmissionLine class. We have to start by
# building the name of the emission line luminosity dataset.
z = 1.4
zString = GH5.getRedshiftString(z)
luminosityName = "diskLineLuminosity:balmerAlpha6563:rest:"+zString
print(luminosityName)

diskLineLuminosity:balmerAlpha6563:rest:z1.4000


The emission line luminosities datasets have the following nomenclature:

    <component>LineLuminosity:<lineName>:<frame>(:<filterName>):z<redshift>(:dust<dustOptions>)
where:

    <component> = the component of the galaxy ('disk', 'spheroid' or 'total')
    <lineName> = the name of the emission line to compute the luminosity for.
    <filterName> = if specified, the luminosity will correspond to the emission line luminosity underneath the
                specified filter (e.g. SDSS r-band), otherwise the luminosity will be the luminsoity one would 
                measure from the line itself.
    <frame> = the frame of the luminosity ('rest' or 'observed')
    <redshift> = the redshift of the snapshot output in which this galaxy is found (e.g 1.4000). Note that for
                lightcones this may be different to the redshift of the galaxy in the lightcone.
    <dustOptions> = if the luminosity has attenuated by interstellar this will consist of a string of commands
                describing the dust method that has been applied.

In [11]:
# First we will check that we can compute this emission line. Computing the line luminosity depends on 
# whether the Galacticus HDF5 file contains luminosities for the Lyman, Helium and Oxygen continuua.
assert(EMLINE.ionizingContinuuaAvailable(luminosityName))

In [9]:
# Compute the emission line luminosity and print top 10 values
lum = EMLINE.getLineLuminosity(luminosityName)
print(lum[:10])

[ 294305.24019514  294305.24019514   67652.97317594   67652.97317594
   26886.76584619   26886.76584619    8377.24153291    8377.24153291
   11782.21428853   11782.21428853]


In [10]:
# We can compute the line luminosity for the entire galaxy using the getLineLuminosity function. The total
# line luminosity is simply equal to the sum of the line luminosities for the disk and bulge.
diskName = "diskLineLuminosity:balmerAlpha6563:rest:"+zString
sphereName = "spheroidLineLuminosity:balmerAlpha6563:rest:"+zString
totalName = "totalLineLuminosity:balmerAlpha6563:rest:"+zString
lum = EMLINE.getLineLuminosity(totalName)
print(lum[:10])
lum = EMLINE.getLineLuminosity(diskName) + EMLINE.getLineLuminosity(sphereName) 
print(lum[:10])

[ 294305.24019514  294305.24019514   67652.97317594   67652.97317594
   26886.76584619   26886.76584619    8377.24153291    8377.24153291
   11782.21428853   11782.21428853]
[ 294305.24019514  294305.24019514   67652.97317594   67652.97317594
   26886.76584619   26886.76584619    8377.24153291    8377.24153291
   11782.21428853   11782.21428853]


In [12]:
GH5.close()